In [ ]:
import os, re, subprocess
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from fpdf import FPDF
from pathvalidate import sanitize_filename
report = r"Q:\2020-2915-00\_construction\_16_material_testing_rpts\_contract_3_cwwtp\Concrete_Test_Results\Concrete_Tracking_Report\C3_concrete_test_result_sheet_20220216.xls"
xls = pd.ExcelFile(report)
pattern = r"^(.*) - (.*) - (\d{2}) [\w]{3} @ (\d{1,2}) [Dd]ays"

In [ ]:
#PDF Class
class PDF(FPDF):
    def header(self):
        # Logo
        self.image(r"AE Logo lores.jpg", 10, 8, 33)
        # Arial bold 15
        self.set_font('Times', 'B', 20)
        # Move to the right
        self.cell(80)
        # Title
        self.cell(30, 10, 'Concrete Test Report Summary', 0, 0, 'C')
        self.cell(80)
        self.set_font('Times', '', 11)
        self.cell(0, 4, 'Arun Kishore', 0, 1, 'R')
        self.cell(0, 4, datetime.now().strftime('%B %d, %Y'), 0, 0, 'R')
        # Line break
        self.ln(20)

    # Page footer
    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        # Arial italic 8
        self.set_font('Times', 'I', 8)
        # Page number
        self.cell(0, 10, 'Page ' + str(self.page_no()) + '/{nb}', 0, 0, 'C')

    def draw_table(self, header, data, cell_width, heading='', subheading=''):
        """Draws a table in PDF based on provided input

        Args:
            pdf (<FPDF class>): _description_
            header (tuple): A tuple with header row values
            data (Array of Arrays): Array with table data
            cell_width (Array): Array of int with col widths
            heading (str, optional): Table Title. Defaults to ''.
            subheading (str, optional): Table subtitle. Defaults to ''.
        """
        #print(header)
        #print(data)
        effective_page_width = self.w - 2*self.l_margin
        cell_offset = (effective_page_width - sum(cell_width))/2

        if heading != '':
            self.set_font('Times', 'B', 14)
            self.cell(w=0, h=5, txt=heading, ln=1, align='C')

        if subheading != '':
            self.set_font('Times', 'I', 10)
            self.cell(w=0, h=4, txt=subheading, ln=1, align='C')

        if cell_offset > 0:
            self.cell(cell_offset)

        self.set_font('Times', 'B', 12)
        for i, datum in enumerate(header):
            self.cell(cell_width[i], self.font_size * 1.5, datum, border=1,align='C')
        self.ln(self.font_size * 1.5)

        self.set_font('Times', '', 12)
        for row in data:
            if cell_offset > 0:
                self.cell(cell_offset)
            for i, datum in enumerate(row):
                self.cell(cell_width[i], self.font_size * 1.5, str(datum), border=1,align='C')
            self.ln(self.font_size * 1.5)
        self.ln(self.font_size * 1.5)
    


In [ ]:
# #Loop through different sheets
# sheet_status = []
# sheets = []
# for sheet_name in xls.sheet_names:
#     try:
#         #sheet_name = xls.sheet_names[2]
#         df = pd.read_excel(report, sheet_name=sheet_name,header = None)
#         match = re.match(pattern, df[0][2])

#         if match == None:
#             sheet_status.append({
#                 'name':sheet_name,
#                 'status': 'Skipped'
#             })
#             #print(" - Skipped")
#             continue

#         sheet_info = {
#             'name':sheet_name,
#             'columns': len(df.columns),
#             'rows': len(df),
#             'mix': match.group(1),
#             'mix-desc': match.group(2),
#             'target_str': int(match.group(3)),
#             'days': int(match.group(4)),
#             'header': df[df[0].str.find('Set') > -1].index[0],
#             'sets':[],
#             'df':df
#         }

#         set_start = df[df.apply(check_if_float, axis=1)].index.to_list()
#         #Loop through different sets
#         for i, start in enumerate(set_start):
#             set_data = {'num': df[0][start]}
#             if i == len(set_start)-1:
#                 end = len(df) - 1
#             else:
#                 end = set_start[i+1]-1

#             sub = df[start:end+1].dropna(how='all')
#             req_data = sub[sub[8] == sheet_info['days']]
#             if len(req_data) == 0:
#                 set_data['Analysis'] = 'Missing Testing day = str required day'
#             elif sub[sub[8] == sheet_info['days']][9].isnull().values.any():
#                 if datetime.today() < sub[sub[8] == sheet_info['days']][7].min():
#                     set_data['Analysis'] = 'Future Test.'
#                 else:
#                     set_data['Analysis'] = 'CRITICAL. Missing test data for past tests.'
#             else:
#                 set_data['Achieved str'] = req_data[9].min()
#                 if req_data[9].min() < sheet_info['target_str']:
#                     set_data['Analysis'] = 'Failed'
#                 else:
#                     set_data['Analysis'] = 'Passed'
            
#             sheet_info['sets'].append(set_data)
#         sheets.append(sheet_info)
#         sheet_status.append({
#             'name':sheet_name,
#             'status': 'Analysed'
#         })
#     except Exception as e:
#         sheet_status.append({
#             'name':sheet_name,
#             'status': 'Error',
#             'error': str(e)
#         })

# required_str = []
# achieved_str = []
# x_axis = []
# for i, set in enumerate(sheet_info['sets']):
#     if set['Analysis'] in ['Passed', 'Failed']:
#         required_str.append(sheet_info['target_str'])
#         achieved_str.append(set['Achieved str'])
#         x_axis.append(str(set['num']))


# plt.bar(x_axis, required_str, color='r', label='Target Str.')
# plt.bar(x_axis, achieved_str, width=0.8, color='b', alpha=0.5, label='Achiveved Str.')
# plt.legend()
# plt.xlabel('Set #')
# plt.ylabel('MPa')
# plt.title("How to read: You shouldn't see any red in graph", fontsize=8)
# plt.suptitle(f'{sheet_info["mix"]} - {sheet_info["mix-desc"]} conc. test summary')
# plt.show()

# def check_if_float(row):
#     return isinstance(row[0], int) and row[0] >0

# set_start = df[df.apply(check_if_float, axis=1)].index.to_list()
# for i, start in enumerate(set_start):
#     if i == len(set_start)-1:
#         end = len(df) - 1
#     else:
#         end = set_start[i+1]-1

#     sub = df[start:end+1].dropna(how='all')
#     req_data = sub[sub[8] == 56]
#     str_achived = req_data[9].mean()
#     sub[sub[8] == 28][9].mean()
    
#     break
# len(sub[sub[8] == 11])

In [ ]:
class Analysis():
    def __init__(self, file):
        self.report = file
        self.pattern = r"^(.*) - (.*) - (\d{2}) [\w]{3} @ (\d{1,2}) [Dd]ays"
        self.sheet_status = []
        self.sheets = []
    
    def parse_xls(self):
        xls = pd.ExcelFile(self.report)
        for sheet_name in xls.sheet_names:
            self.parse_sheet(sheet_name)

        for i, sheet in enumerate(self.sheets):
            self.sheets[i]['target_vs_achieved'] = self.plot_targetVachieved(sheet)
            
    def check_if_float(self, row):
        return isinstance(row[0], int) and row[0] >0

    def parse_sheet(self, sheet_name):
        try:
            df = pd.read_excel(self.report, sheet_name=sheet_name,header = None)
            match = re.match(self.pattern, df[0][2])

            if match == None:
                self.sheet_status.append({
                    'name':sheet_name,
                    'status': 'Skipped'
                })
                return

            sheet_info = {
                'name':sheet_name,
                'columns': len(df.columns),
                'rows': len(df),
                'mix': match.group(1),
                'mix-desc': match.group(2),
                'target_str': int(match.group(3)),
                'days': int(match.group(4)),
                'header': df[df[0].str.find('Set') > -1].index[0],
                'sets':[],
                'df':df
            }

            set_start = df[df.apply(self.check_if_float, axis=1)].index.to_list()

            for i, start in enumerate(set_start):
                end = len(df) - 1 if i == len(set_start)-1 else set_start[i+1]-1
                sub = df[start:end+1].dropna(how='all')
                set_data = {
                    'num': df[0][start],
                    'df': sub
                }

                req_data = sub[sub[8] == sheet_info['days']]
                
                if len(req_data) == 0:
                    set_data['Analysis'] = 'Missing Testing day = str required day'
                    
                elif sub[sub[8] == sheet_info['days']][9].isnull().values.any():
                    if datetime.today() < sub[sub[8] == sheet_info['days']][7].min():
                        set_data['Analysis'] = 'Future Test.'
                    else:
                        set_data['Analysis'] = 'CRITICAL. Missing test data for past tests.'
                else:
                    set_data['Achieved str'] = req_data[9].min()
                    if req_data[9].min() < sheet_info['target_str']:
                        set_data['Analysis'] = 'Failed'
                    else:
                        set_data['Analysis'] = 'Passed'
                
                sheet_info['sets'].append(set_data)
                
            self.sheets.append(sheet_info)
            self.sheet_status.append({
                'name':sheet_name,
                'status': 'Analysed'
            })

        except Exception as e:
            self.sheet_status.append({
                'name':sheet_name,
                'status': 'Error',
                'error': str(e)
                })

    def plot_targetVachieved(self, sheet):
        required_str = []
        achieved_str = []
        x_axis = []
        filename = sanitize_filename(sheet['name'] + '.png')
        
        for i, set in enumerate(sheet['sets']):
            if set['Analysis'] in ['Passed', 'Failed']:
                required_str.append(sheet['target_str'])
                achieved_str.append(set['Achieved str'])
                x_axis.append(str(set['num']))

        if x_axis == []:
            return None
        plt.clf()
        plt.bar(x_axis, required_str, color='r', label='Target Str.')
        plt.bar(x_axis, achieved_str, width=0.8, color='b', alpha=0.5, label='Achiveved Str.')
        plt.legend()
        plt.xlabel('Set #')
        plt.ylabel('MPa')
        plt.title("How to read: You shouldn't see any red in graph", fontsize=8)
        plt.suptitle(f'{sheet["mix"]} - {sheet["mix-desc"]} conc. test summary')
        plt.savefig(filename)
        return filename

In [ ]:
xl1 = Analysis(report)
xl1.parse_xls()

# PDF REPORT

#### PDF Functions

In [ ]:
# Instantiation of inherited class
pdf = PDF()
pdf.alias_nb_pages()
pdf.add_page()
effective_page_width = pdf.w - 2*pdf.l_margin

sheets = xl1.sheets
sheet_status = xl1.sheet_status
summary = f"The below analysis is based on information from '{os.path.basename(report)}'. For this analysis, a total of {len(sheet_status)} sheets were considered, out of which {len([x for x in sheet_status if x['status']=='Analysed'])} were deemed to have the data necessary for analsysis."
sheet_status_data =[]

for sh in xl1.sheet_status:
    name = sh['name']
    if sh['status'] != 'Analysed':
        status = 'Skipped'
        num_sets = 'N/A'
    else:
        status = 'Analysed'
        for sh in sheets:
            if sh['name'] == name:
                num_sets = len(sh['sets'])
        
    
    sheet_status_data.append((name, status, num_sets))

#Summary
pdf.set_font('Times', 'B', 18)
pdf.cell(w=0, h=10, txt='Summary:', border=0, ln=1, fill=False)
pdf.set_font('Times', '', 12)
pdf.multi_cell(w=0, h=6, txt=summary, border=0, fill=False)
pdf.ln(pdf.font_size * 2.5)

#Sheet Summary Table
width = (effective_page_width / 3 + 20,effective_page_width / 3 - 10,effective_page_width / 3 - 10)
pdf.draw_table(('Sheet Name', 'Status', 'Sets Considered'), sheet_status_data, width)

#Sets that need consideration
passed_data, failed_data, missing_test_data, missing_day_data, future_data = [], [], [], [], []
for sh in sheets:
    status = {
        'Passed': [],
        'CRITICAL. Missing test data for past tests.': [], 
        'Missing Testing day = str required day': [], 
        'Future Test.': [],
        'Failed': []
    }

    for set in sh['sets']:
        status[set['Analysis']].append(str(set['num']))
    
    #if status['CRITICAL. Missing test data for past tests.'] != []:
    missing_status = status['CRITICAL. Missing test data for past tests.']
    if len(missing_status) > 15:
        txt = ', '.join(missing_status[:10]) + f", + {5+len(missing_status) - 10} more"
        missing_test_data.append([sh['name'], txt])
    elif len(missing_status) > 0:
        txt = ', '.join(missing_status)
        missing_test_data.append([sh['name'], txt])

    testday_status = status['Missing Testing day = str required day']
    if status['Missing Testing day = str required day'] != []:
        if len(testday_status) > 15:
            txt = ', '.join(testday_status[:10]) + f", + {5+len(testday_status) - 10} more"
        else:
            txt = ', '.join(testday_status)
        missing_day_data.append([sh['name'], txt])

    if status['Future Test.'] != []:
        if len(status['Future Test.']) > 15:
            txt = ', '.join(status['Future Test.'][:10]) + f", + {5+len(status['Future Test.']) - 10} more"
        else:
            txt = ', '.join(status['Future Test.'])
        future_data.append([sh['name'], txt])

    if status['Failed'] != []:
        if len(status['Failed']) > 15:
            txt = ', '.join(status['Failed'][:10]) + f", + {5+len(status['Failed']) - 10} more"
        else:
            txt = ', '.join(status['Failed'])
        failed_data.append([sh['name'], txt])
    

pdf.add_page()
pdf.set_font('Times', 'B', 18)
pdf.cell(w=0, h=10, txt='Result Summary', border=0, ln=1, fill=False, align='C')
pdf.set_font('Times', '', 12)
pdf.multi_cell(
    w=0, 
    h=6, 
    txt='The following tables list the sets of tests whose results are not optimal.', 
    border=0, 
    fill=False,
    align = 'C'
    )
pdf.ln(pdf.font_size * 1.5)

if failed_data != []:
    pdf.draw_table(
        header=('Mix Name & Desc.', 'Sets'),
        data = failed_data,
        cell_width=[effective_page_width/3, 2*effective_page_width/3],
        heading='Failed Sets',
        subheading="See below for a list of sets that have failed to meet the required strength"
        )

if missing_test_data != []:
    pdf.draw_table(
        header=('Mix Name & Desc.', 'Sets'),
        data = missing_test_data,
        cell_width=[effective_page_width/3, 2*effective_page_width/3],
        heading='Missing Data',
        subheading="Sets missing past test data"
        )


if missing_day_data != []:
    pdf.draw_table(
        header=('Mix Name & Desc.', 'Sets'),
        data = missing_day_data,
        cell_width=[effective_page_width/3, 2*effective_page_width/3],
        heading='Testing day != str. req. day',
        subheading="Sets where proposed test days don't match up with the req. strength days"
        )

pdf.add_page()
pdf.set_font('Times', 'B', 18)
pdf.cell(w=0, h=10, txt='Sheet Summary', border=0, ln=1, fill=False, align='C')
pdf.set_font('Times', '', 12)
pdf.multi_cell(w=0, h=6, txt='The following tables list the summary of tests by sheet', border=0, fill=False, align='C')
pdf.ln(pdf.font_size * 1.5)


for sh in sheets:
    status = {
        'Passed': 0,
        'CRITICAL. Missing test data for past tests.': 0, 
        'Missing Testing day = str required day': 0, 
        'Future Test.': 0,
        'Failed': 0
    }

    for set in sh['sets']:
        status[set['Analysis']] +=1

    table_data = [
        ['Passed', status['Passed'], 'Str. requirements met'],
        ['Failed', status['Failed'], 'Str. requirements not met'],
        ['Missing Data', status['CRITICAL. Missing test data for past tests.'], 'Tests where scheduled test data missing'],
        ['Testing day != str. req. day', status['Missing Testing day = str required day'], 'Testing day do not match req. testing day from italics above'],
        ['Future tests', status['Future Test.'], 'These tests will be performed in future'],
    ]

    pdf.draw_table(
        header=('Status', '# of sets','Remarks'),
        data = table_data,
        cell_width=[60, 20, effective_page_width - 80],
        heading=sh['name'],
        subheading=f"Summary for {sh['mix']} - {sh['mix-desc']}: {sh['target_str']} MPa @ {sh['days']} days"
    )

pdf.add_page()
pdf.set_font('Times', 'B', 18)
pdf.cell(w=0, h=10, txt='Target strength vs Achived strength', border=0, ln=1, fill=False, align='C')
pdf.set_font('Times', '', 12)
pdf.multi_cell(w=0, h=6, txt='The charts plot the target strengths vs achieved strengths.', border=0, fill=False, align='C')
pdf.ln(pdf.font_size * 1.5)
for sh in sheets:
    if sh['target_vs_achieved'] != None:
        pdf.cell(20)
        pdf.image(sh['target_vs_achieved'])
        pdf.ln(pdf.font_size * 2.5)
outputfile, _ = os.path.splitext(report)
outputfile += '-Report.pdf'
pdf.output(outputfile, 'F')
subprocess.Popen(outputfile, shell=True)

del pdf

## Finish

In [ ]:
for sh in sheets:
    if sh['target_vs_achieved'] != None:
        try:
            os.remove(sh['target_vs_achieved'])
        except Exception as e:
            print(e)

del xl1
